In [1]:
#install.packages('caTools') 
#install.packages('e1071') 
#install.packages('lubridate')
#install.packages("kernlab")
#library(lubridate)
#library(kernlab)
#library(caTools)
#library(tidyverse)
#library(caret)
#library(e1071)

In [142]:
#importing cleaned dataset and hash data
df <- readRDS('CleanedData.Rda')
partnerNameHash <- readRDS("PartnerName.Rda")
stateNameHash <- readRDS("StateName.Rda")

In [143]:
#making age group partitions
ageGroup <- c("0-18","19-36","37-54","55 and above")
df$AgeGroup <- cut(df$Age, breaks = c(0,18,36,54, Inf), labels = ageGroup, right = FALSE)

#making likelihood score partitions
likeGroup <- c("Detractors","Passives","Promoters")
df$LikhihoodGroup <- cut(df$LikelihoodRecommendScore, breaks = c(0,7,9, Inf), labels = likeGroup, right = FALSE)

#removing rows with Price Sensitivity =4
df <- df[-which(df$PriceSensitivity==4),] 

#df <- df[-which(df$OriginState=="VT"),]
#df <- df[-which(df$OriginState=="MS"),]
#df <- df[-which(df$OriginState=="NJ"),]
#df <- df[-which(df$DestinationState=="KY"),]
#df <- df[-which(df$DestinationState=="MS"),]
#df <- df[-which(df$DestinationState=="NJ"),]



In [136]:
table(df$OriginState)


  AK   AL   AR   AZ   CA   CO   CT   DC   FL   GA   HI   IA   ID   IL   IN   KS 
  74   59   42  437 1231  559   45  289  723  783   42   28   19  742   87   19 
  LA   MA   MD   ME   MI   MN   MO   MS   MT   NC   ND   NE   NH   NJ   NM   NV 
 134  232  175    6  254  249  209    2   40  251   25   45   17    3   55  345 
  NY   OH   OK   OR   PA   PR   RI   SC   SD   TN   TX   UT   VA   VT   WA   WI 
 538  220   90  122  242   66   27   44   17   62  907  270   38    5  240  129 
  WY 
  31 

In [137]:
#Names of the columns
colnames(df)

[1] "DestinationCity"          "OriginCity"              
 [3] "AirlineStatus"            "Age"                     
 [5] "Gender"                   "PriceSensitivity"        
 [7] "YearOfFirstFlight"        "FLightsPerYear"          
 [9] "Loyalty"                  "TypeOfTravel"            
[11] "TotalFreqFlyAccount"      "ShoppingAmount"          
[13] "FoodExpenses"             "Class"                   
[15] "FlightDate"               "PartnerCode"             
[17] "ScheduleDepHour"          "DepartureDelayInMin"     
[19] "ArrivalDelayInMin"        "FlightCancelledByUser"   
[21] "FlightDuration"           "Distance"                
[23] "LikelihoodRecommendScore" "OriginLong"              
[25] "OriginLat"                "DestLong"                
[27] "DestLat"                  "DestinationState"        
[29] "OriginState"              "AgeGroup"                
[31] "LikhihoodGroup"

In [138]:
#Converting all categorical data into factors
df$TypeOfTravel <- as.factor(df$TypeOfTravel)
df$PartnerCode <- as.factor(df$PartnerCode)
df$AirlineStatus <- as.factor(df$AirlineStatus)
df$AgeGroup <- as.factor(df$AgeGroup)
df$PriceSensitivity <- as.factor(df$PriceSensitivity)
df$Gender <- as.factor(df$Gender)

df$TotalFreqFlyAccount <- as.factor(df$TotalFreqFlyAccount)

In [146]:
df$OriginState <- as.factor(df$OriginState)
df$DestinationState <- as.factor(df$DestinationState)

In [147]:
#Splitting data into training and test set
set.seed(123) 
split = sample.split(df$LikhihoodGroup, SplitRatio = 0.75) 
  
training_set = subset(df, split == TRUE) 
test_set = subset(df, split == FALSE) 

In [150]:
classifier = ksvm(LikhihoodGroup ~ TypeOfTravel+AirlineStatus+AgeGroup+PartnerCode+
                                  Gender+YearOfFirstFlight+PriceSensitivity+Loyalty+
                                  OriginState+DestinationState+TotalFreqFlyAccount,
                 type = 'C-svc', 
                 kernel = 'rbfdot',C=2,cross=2,
                data=training_set) 
y_pred = predict(classifier, newdata = select(test_set,TypeOfTravel,AirlineStatus,
                                              AgeGroup,PartnerCode,Gender,YearOfFirstFlight,
                                              PriceSensitivity,Loyalty,OriginState,
                                              DestinationState,TotalFreqFlyAccount)) 
cm = table(test_set[, 31], y_pred) 

In [151]:
confusionMatrix(cm)

Confusion Matrix and Statistics

            y_pred
             Detractors Passives Promoters
  Detractors        573       89       153
  Passives          216      204       396
  Promoters          31       99       806

Overall Statistics
                                          
               Accuracy : 0.6167          
                 95% CI : (0.5975, 0.6355)
    No Information Rate : 0.5279          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.4171          
                                          
 Mcnemar's Test P-Value : < 2.2e-16       

Statistics by Class:

                     Class: Detractors Class: Passives Class: Promoters
Sensitivity                     0.6988         0.52041           0.5948
Specificity                     0.8615         0.71862           0.8927
Pos Pred Value                  0.7031         0.25000           0.8611
Neg Pred Value                  0.8590         0.89263    